In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import pandas as pd
import time
import html.parser
from urllib.parse import urlsplit

In [2]:
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless = False)

In [3]:
url = "https://www.golfdigest.com/gallery/americas-100-greatest-golf-courses-ranking"
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, "html.parser")

In [17]:
#testing code to make sure I'm scraping the right data points
#rank = soup.find("span", class_="listicle-counter index").text
#name = soup.find("h2", class_="listicle-title").text
#caption = soup.find("div", class_= "listicle-caption").text
#print(f"rank: {rank}")
#print(f"name: {name}")
#print(f"caption: {caption}")

In [5]:
#loop through each div we are looking to scrape data from
results = soup.find_all("div", class_="listicle-item")

In [6]:
#create empty lists to store scraped data
ranks = []
names = []
captions = []

for result in results:
    # Identify and return rank of course
    rank = result.find("span", class_="listicle-counter index").text
   # Identify and return name of course
    name = result.find("h2", class_="listicle-title").text
    # Identify and return additional info about course
    caption = result.find("div", class_= "listicle-caption").text

    # add results to empty lists
    ranks.append(rank)
    names.append(name)
    captions.append(caption)


In [7]:
golf_df = pd.DataFrame(list(zip(ranks, names, captions)), columns =["Rank", "Course Name", "Caption"])
golf_df

Rank            Course Name  \
0     1       PINE VALLEY G.C.   
1     2  AUGUSTA NATIONAL G.C.   
2     3     CYPRESS POINT CLUB   
3     4  SHINNECOCK HILLS G.C.   
4     5           OAKMONT C.C.   
..  ...                    ...   
95   96      THE PRESERVE G.C.   
96   97  BLACKWOLF RUN (River)   
97   98          OLD TOWN CLUB   
98   99     CROOKED STICK G.C.   
99  100          MAYACAMA G.C.   

                                              Caption  
0   7,181 yards, Par 70 / Points: 72.0078 \nPine V...  
1   7,435 yards, Par 72 / Points: 71.5810Augusta, ...  
2   6,524 yards, Par 72 / Points: 69.6830 Alister ...  
3   7,445 yards, Par 70 / Points: 69.2526William F...  
4   7,255 Yards, Par 71 / Points: 68.9218Oakmont, ...  
..                                                ...  
95  7,138 Yards, Par 72\t/ Points: 61.0619Carmel, ...  
96  7,404 Yards, Par\t72\t/ Points: 61.0451\nKohle...  
97  7,038 Yards, Par 70 / Points: 61.0033\nWinston...  
98  7,516 Yards, Par \t72\t/ Points: 60.9908 Carme...  
99  6,761 Yards, Par 71 / Points: 60.9746 \nSanta ...  

[100 rows x 3 columns]

In [8]:
#import csv of lat long of courses
locations = "lat_long.csv"
locations_df = pd.read_csv(locations, encoding="ISO-8859-1")
locations_df.head()

Course Name   Latitude   Longitude
0       PINE VALLEY G.C.  39.792235  -74.972227
1  AUGUSTA NATIONAL G.C.  33.502354  -82.022577
2     CYPRESS POINT CLUB  36.580509 -121.974148
3  SHINNECOCK HILLS G.C.  40.894252  -72.439777
4           OAKMONT C.C.  40.526148  -79.826869

In [9]:
#merge data frames on course name
df_merge = pd.merge(golf_df, locations_df, on='Course Name')

df_merge

Rank            Course Name  \
0     1       PINE VALLEY G.C.   
1     2  AUGUSTA NATIONAL G.C.   
2     3     CYPRESS POINT CLUB   
3     4  SHINNECOCK HILLS G.C.   
4     5           OAKMONT C.C.   
..  ...                    ...   
93   96      THE PRESERVE G.C.   
94   97  BLACKWOLF RUN (River)   
95   98          OLD TOWN CLUB   
96   99     CROOKED STICK G.C.   
97  100          MAYACAMA G.C.   

                                              Caption   Latitude   Longitude  
0   7,181 yards, Par 70 / Points: 72.0078 \nPine V...  39.792235  -74.972227  
1   7,435 yards, Par 72 / Points: 71.5810Augusta, ...  33.502354  -82.022577  
2   6,524 yards, Par 72 / Points: 69.6830 Alister ...  36.580509 -121.974148  
3   7,445 yards, Par 70 / Points: 69.2526William F...  40.894252  -72.439777  
4   7,255 Yards, Par 71 / Points: 68.9218Oakmont, ...  40.526148  -79.826869  
..                                                ...        ...         ...  
93  7,138 Yards, Par 72\t/ Points: 61.0619Carmel, ...  36.534669 -121.868539  
94  7,404 Yards, Par\t72\t/ Points: 61.0451\nKohle...  43.727082  -87.775916  
95  7,038 Yards, Par 70 / Points: 61.0033\nWinston...  36.124647  -80.271688  
96  7,516 Yards, Par \t72\t/ Points: 60.9908 Carme...  39.946528  -86.193029  
97  6,761 Yards, Par 71 / Points: 60.9746 \nSanta ...  38.528184 -122.733709  

[98 rows x 5 columns]

In [14]:
df_merge.to_csv("full_data_set.csv", index=False, header=True)

In [13]:
description = df_merge.Caption
description

0     7,181 yards, Par 70 / Points: 72.0078 \nPine V...
1     7,435 yards, Par 72 / Points: 71.5810Augusta, ...
2     6,524 yards, Par 72 / Points: 69.6830 Alister ...
3     7,445 yards, Par 70 / Points: 69.2526William F...
4     7,255 Yards, Par 71 / Points: 68.9218Oakmont, ...
                            ...                        
93    7,138 Yards, Par 72\t/ Points: 61.0619Carmel, ...
94    7,404 Yards, Par\t72\t/ Points: 61.0451\nKohle...
95    7,038 Yards, Par 70 / Points: 61.0033\nWinston...
96    7,516 Yards, Par \t72\t/ Points: 60.9908 Carme...
97    6,761 Yards, Par 71 / Points: 60.9746 \nSanta ...
Name: Caption, Length: 98, dtype: object